# Load Packages

In [1]:
import os
import cv2
import math
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from glob import glob
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import shutil
#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

# File Paths

In [55]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"
tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
#https://stackoverflow.com/questions/5953373/how-to-split-image-into-multiple-pieces-in-python
correct_directory = os.path.join(parent_directory,"temp/rechip")
incorrectly_chipped_images_path = os.path.join(parent_directory,"temp/incorrectly_chipped_images")
remaining_chips_path =  os.path.join(parent_directory,"temp/remaining_chips")
remaining_sets_path =  os.path.join(parent_directory,"temp/remaining_sets")

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

In [59]:
print("number of remaining chips", len(os.listdir(remaining_chips_path)))

number of remaining chips 193991


In [58]:
#get a numpy array of the tiles corresponding remaining chips lists
tiles = []
remaining_chips_list = os.listdir(remaining_chips_path) #get a list of the remaining chip names 
for image_name in os.listdir(remaining_chips_path):
    tiles.append(os.path.splitext(image_name)[0].rsplit("_",2)[0])
tiles = np.unique(tiles) #get a list of the tile names corresponding to the remaining chips

#create counters
no_tiles_in_set = 0
number_of_images = 0
all_image_count = 0 
#create set paths
set_no = 1
set_path = os.path.join(remaining_sets_path,"set_"+ str(set_no))
os.makedirs(set_path, exist_ok = True)
#state the number of images that need to be moved 
print("number of remaining chips", len(os.listdir(remaining_chips_path)))

for tile_name in tqdm.tqdm(tiles):
    #get images corresponding to each tile
    images_in_tile = [string for string in remaining_chips_list if tile_name in string]
    
    #move images for tile if there are less than 8,000 images in the current set
    if len(os.listdir(set_path)) < 8000:
        no_tiles_in_set += 1
        number_of_images += len(images_in_tile)
        #move images
        for image in images_in_tile: #copy images
            shutil.copyfile(os.path.join(remaining_chips_path, image),
                            os.path.join(set_path, image)) 
            all_image_count += 1
    
    else: #if there are over 8,000 images, move the current images, and create next set 
        no_tiles_in_set += 1
        number_of_images += len(images_in_tile)
        for image in images_in_tile: #copy images
            shutil.copyfile(os.path.join(remaining_chips_path, image),
                            os.path.join(set_path, image))
            all_image_count += 1
            
        print(set_no, number_of_images, no_tiles_in_set)
        #reset counters
        no_tiles_in_set = 0
        number_of_images = 0
        
        #create next set directory
        set_no += 1
        set_path = os.path.join(remaining_sets_path,"set_"+ str(set_no))
        os.makedirs(set_path, exist_ok = True)
print("images allocated to sets", all_image_count)

number of remaining chips 72


  7%|▋         | 152/2136 [05:30<1:14:11,  2.24s/it]

1 8126 152


 13%|█▎        | 277/2136 [12:25<2:12:50,  4.29s/it]

2 8155 125


...

22 8224 67


 98%|█████████▊| 2087/2136 [2:18:08<05:31,  6.77s/it]

23 8155 64


100%|██████████| 2136/2136 [2:23:11<00:00,  4.02s/it]

images allocated to sets 193991
